## FDTree Class

In [1]:
import numpy as np
import pickle
from scipy.sparse import find

class DecisionTreeModel:
    def __init__(self, source, depth_threshold=10, plambda=7, MSP_item=200):
        '''
            sMatrix: I*U matrix
            depth_threshold: terminate depth
            plambda: regularization parameter
            self.rI: dict { itemid1, itemid2, itemid3 ... }
            self.rU: dict {
                        userid1: { itemid11: rating11, itemid12: rating12, ... } rating of user 1
                        userid2: { itemid21: rating21, itemid22: rating22, ... } rating of user 2
                        userid3: ...
                     }
            self.lr_bound: dict {
                                level 0: [[left_bound, right_bound]], users' bound for one level, each ele in dictionary represents one node
                                level 1: [[left_bound, right_bound], [left_bound, right_bound], [left_bound, right_bound]], 3
                                level 2: ..., 9
                            } (bound means index)
            self.tree: []  all of userid
            self.split_item: list [
                    level 0: []
                    level 1: []
                    level 2: []
            ]
            self.sum_cur_t: dict {
                        itemid1: {'rating': sum of ratings for item 1, 'cnt': sum of users rated item 1}
                        itemid2: {'rating': sum of ratings for item 2, 'cnt': sum of users rated item 2}
                        ...
                    }
            self.sum_2_cur_t: dict {
                        itemid1: sum of square ratings for item 1
                        itemid2: sum of square ratings for item 2
                        ...
                    }
            self.biasU: dict {
                        userid1: bias1
                        userid2: bias2
                        ...
                    }
            every element represents ratings for one item, its order decide the users in tree nodes
        '''
        self.real_item_num = source.shape[0]
        x = find(source)
        itemset = x[0]
        userset = x[1]
        self.rU = {}
        self.global_mean = 0  # global average of ratings   
        self.depth_threshold = depth_threshold
        self.plambda = plambda
        self.cur_depth = 0
        self.MSP_item = MSP_item

        #### Calculate rate of progress ####
        self.node_num = 0
        self.cur_node = 0
        for i in range(self.depth_threshold):
            self.node_num += 3 ** i

        #### Generate rI, rU ####
        print("rU Generation start:")
        self.rI = list(set(source.nonzero()[0]))
        i = 0
        num_ratings = len(userset)
        for itemid, userid in zip(itemset, userset):
            if i % 10000 == 0:
                print("%.2f%%" % (100 * i / num_ratings))
            i += 1
            self.rU.setdefault(userid, {})[itemid] = source[itemid, userid]
            self.global_mean += source[itemid, userid]
        self.global_mean /= len(itemset)
        self.item_size = len(self.rI)
        self.user_size = len(self.rU)
        print("rU Generation DONE")

        #### Initiate Tree, lr_bound ####
        self.tree = list(self.rU.keys())
        self.split_item = []
        self.lr_bound = {'0': [[0, len(self.tree) - 1]]}

        #### Generate bias, sum_cur_t, sum_2_cur_t ####
        self.biasU = {}
        self.sum_cur_t = np.zeros(self.real_item_num)
        self.sum_2_cur_t = np.zeros(self.real_item_num)
        self.sum_cntt = np.zeros(self.real_item_num)
        print("bias, sum_cur_t, sum_2_cur_t Generation start:")
        i = 0
        for userid in self.rU:
            if i % 5000 == 0:
                print("%.2f%%" % (100 * i / (0.7 * 480189)))
            i += 1
            self.biasU[userid] = (sum(list(self.rU[userid].values())) \
                                     + self.plambda * self.global_mean) /   \
                                 (self.plambda + len(self.rU[userid]))
            user_all_rating_id = np.array(list(self.rU[userid].keys()))
            # print('user_all_rating_id ', user_all_rating_id[:])
            user_all_rating = np.array(list(self.rU[userid].values()))
            self.sum_cur_t[user_all_rating_id[:]] += user_all_rating[:] - self.biasU[userid]
            self.sum_2_cur_t[user_all_rating_id[:]] += (user_all_rating[:] - self.biasU[userid]) ** 2
            self.sum_cntt[user_all_rating_id[:]] += 1
        print("bias, sum_cur_t, sum_2_cur_t Generation DONE")

        print("Initiation DONE!")

    def calculate_error(self, sumt, sumt_2, cntt):
        ''' Calculate error for one item-split in one node '''
        Error_i = np.sum(sumt_2 - (sumt ** 2) / (cntt + 1e-9))

        return Error_i

    def generate_decision_tree(self, lr_bound_for_node, chosen_id):
        '''
            sumtL: dict {
                itemid1: {'rating': sum of ratings for item 1, 'cnt': sum of users rated item 1}
                itemid2: {'rating': sum of ratings for item 2, 'cnt': sum of users rated item 2}
                ...
            }
            sumtL_2: dict {
                itemid1: sum of square ratings for item 1
                itemid2: sum of square ratings for item 2
                ...
            }
            lr_bound_for_node: list [leftind, rightind] for one node
        '''

        #### Terminate ####
        self.cur_depth += 1
        if self.cur_depth > self.depth_threshold or len(chosen_id) == self.item_size:
            return

        #### Choose Most Popular Items of This Node ####
        num_rec = np.zeros(self.real_item_num)
        for userid in self.tree[lr_bound_for_node[0]:(lr_bound_for_node[1] + 1)]:
            user_all_rating_id = np.array(list(self.rU[userid].keys()))
            num_rec[user_all_rating_id[:]] += 1
        sub_item_id = np.argsort(num_rec)[:self.MSP_item]

        #### Find optimum item to split ####
        min_sumtL, min_sumtD, min_sumtL_2, min_sumtD_2, min_sumtU, min_sumtU_2, Error = {}, {}, {}, {}, {}, {}, {}
        min_Error = "None"
        for itemid in sub_item_id:
            if itemid in chosen_id:
                continue
            '''
                user_rating_item_in_nodet: [ [uid01, rating01], [uid02, rating02], ... ]
                to find all users in node t who rates item i
            '''
            user_rating_item_in_nodet = ([userid, self.rU[userid][itemid]] for userid in
                                         self.tree[lr_bound_for_node[0]:(lr_bound_for_node[1] + 1)] if
                                         itemid in self.rU[userid])
            sumt = np.zeros((self.real_item_num, 3))
            sumt_2 = np.zeros((self.real_item_num, 3))
            cntt = np.zeros((self.real_item_num, 3))
            for user in user_rating_item_in_nodet:
                ''' user_all_rating: array [ [itemid11, rating11], [itemid12, rating12], ... ] '''
                user_all_rating_id = np.array(list(self.rU[user[0]].keys()))
                user_all_rating = np.array(list(self.rU[user[0]].values()))
                #### calculate sumtL for node LIKE ####
                if user[1] >= 4:
                    sumt[user_all_rating_id[:], 0] += user_all_rating[:] - self.biasU[user[0]]
                    sumt_2[user_all_rating_id[:], 0] += (user_all_rating[:] - self.biasU[user[0]]) ** 2
                    cntt[user_all_rating_id[:], 0] += 1
                #### calculate sumtD for node DISLIKE ####
                elif user[1] <= 3:
                    sumt[user_all_rating_id[:], 1] += user_all_rating[:] - self.biasU[user[0]]
                    sumt_2[user_all_rating_id[:], 1] += (user_all_rating[:] - self.biasU[user[0]]) ** 2
                    cntt[user_all_rating_id[:], 1] += 1
            #### calculate sumtU for node UNKNOWN ####
            sumt[:, 2] = self.sum_cur_t[:] - sumt[:, 0] - sumt[:, 1]
            sumt_2[:, 2] = self.sum_2_cur_t[:] - sumt_2[:, 0] - sumt_2[:, 1]
            cntt[:, 2] = self.sum_cntt[:] - cntt[:, 0] - cntt[:, 1]
            Error[itemid] = self.calculate_error(sumt, sumt_2, cntt)

            if min_Error == "None" or Error[itemid] < min_Error:
                min_sumt = sumt
                min_sumt_2 = sumt_2
                min_cntt = cntt
                min_Error = Error[itemid]
        #### Find optimum split-item ####
        optimum_itemid = min(Error, key=Error.get)
        if len(self.split_item) == self.cur_depth - 1:
            self.split_item.append([optimum_itemid])
        else:
            self.split_item[self.cur_depth - 1].append(optimum_itemid)
        # self.split_item.setdefault(str(self.cur_depth-1), []).append(optimum_itemid)
        chosen_id.append(optimum_itemid)

        #### sort tree ####
        self.lr_bound.setdefault(str(self.cur_depth), []).append([])  # for LIKE
        self.lr_bound[str(self.cur_depth)].append([])  # for DISLIKE
        self.lr_bound[str(self.cur_depth)].append([])  # for UNKNOWN
        listU, listL, listD = [], [], []
        for userid in self.tree[lr_bound_for_node[0]:(lr_bound_for_node[1] + 1)]:
            if optimum_itemid not in self.rU[userid]:
                listU.append(userid)
            elif self.rU[userid][optimum_itemid] >= 4:
                listL.append(userid)
            elif self.rU[userid][optimum_itemid] <= 3:
                listD.append(userid)
        self.tree[lr_bound_for_node[0]:(lr_bound_for_node[1] + 1)] = listL + listD + listU
        self.lr_bound[str(self.cur_depth)][-3] = [lr_bound_for_node[0],
                                                  lr_bound_for_node[0] + len(listL) - 1]  # for LIKE
        self.lr_bound[str(self.cur_depth)][-2] = [lr_bound_for_node[0] + len(listL),
                                                  lr_bound_for_node[0] + len(listL) + len(listD) - 1]  # for DISLIKE
        self.lr_bound[str(self.cur_depth)][-1] = [lr_bound_for_node[0] + len(listL) + len(listD),
                                                  lr_bound_for_node[0] + len(listL) + len(listD) + len(
                                                      listU) - 1]  # for UNKNOWN

        #### Generate Subtree of Node LIKE ####
        self.sum_cur_t = min_sumt[:, 0]
        self.sum_2_cur_t = min_sumt_2[:, 0]
        self.sum_cntt = min_cntt[:, 0]
        self.generate_decision_tree(self.lr_bound[str(self.cur_depth)][-3], chosen_id[:])
        self.cur_depth -= 1

        #### Generate Subtree of Node DISLIKE ####
        self.sum_cur_t = min_sumt[:, 1]
        self.sum_2_cur_t = min_sumt_2[:, 1]
        self.sum_cntt = min_cntt[:, 1]
        self.generate_decision_tree(self.lr_bound[str(self.cur_depth)][-2], chosen_id[:])
        self.cur_depth -= 1

        #### Generate Subtree of Node UNKNOWN ####
        self.sum_cur_t = min_sumt[:, 2]
        self.sum_2_cur_t = min_sumt_2[:, 2]
        self.sum_cntt = min_cntt[:, 2]
        self.generate_decision_tree(self.lr_bound[str(self.cur_depth)][-1], chosen_id[:])
        self.cur_depth -= 1

        #### Show Rating Progress ####
        for i in range(self.cur_depth - 1):
            print("┃", end="")
        print("┏", end="")
        self.cur_node += 1
        print("Current depth: " + str(self.cur_depth) + "        %.2f%%" % (100 * self.cur_node / self.node_num))

    def build_model(self):
        #### Construct the tree & get the prediction model ####
        self.generate_decision_tree(self.lr_bound['0'][0], [])

## Step 1: Input Dataset

In [2]:
from scipy.sparse import load_npz

In [6]:
############### Load Data ##################
rating_matrix_csc = load_npz('netflix/sparse_matrix_100%.npz').tocsc()
print("file load DONE")
############################################

file load DONE


In [ ]:
############### Build Tree #################
''' Save to file 'tree.pkl' '''
start = 0
end = int(rating_matrix_csc.shape[1] * 0.75)
dtmodel_realdata = dt.DecisionTreeModel(rating_matrix_csc[:, start:end], depth_threshold = 10)
dtmodel_realdata.build_model()
o_tree = open('./tree_data_structure/tree.pkl', 'wb')
pickle.dump(dtmodel_realdata, o_tree)
o_tree.close()
############################################

In [ ]:

######################## Build Predict Model #########################
plambda_candidates = {}
inputf = open('./tree_data_structure/tree.pkl', 'rb')
dtmodel_realdata = pickle.load(inputf)
for level in dtmodel_realdata.lr_bound:
	plambda_candidates[level] = list(np.arange(0.001, 0.05, 0.0005))
prediction_model = tf.generate_prediction_model(dtmodel_realdata, plambda_candidates, rating_matrix_csc[:, endt:end])
######################################################################

######################### Test for New-user ##########################
rmse_result = tf.pred_RMSE_for_new_user(dtmodel_realdata, prediction_model, rating_matrix_csc[:, end:])
######################################################################



    

In [9]:
import pandas as pd
from scipy.sparse import *
catagory = "Grocery_and_Gourmet_Food"
item_data = 'item_metadata/meta_' + catagory + '.csv'
rating_data = 'user_ratings/' + catagory + '.csv'
# print(rating_data)
ratingsFrame = pd.read_csv(rating_data, names = ["userID", "itemID", "rating"])
ratingsFrame.sort_values(by = 'userID', ascending=True, inplace=True)
itemsFrame = pd.read_csv(item_data)
item_lst = ratingsFrame["itemID"].tolist()
user_lst = ratingsFrame["userID"].tolist()
rating_lst = ratingsFrame["rating"].tolist()
row = []
col = []
rating_data = []
for i in range(ratingsFrame.shape[0]):
    row.append(item_lst[i])
    col.append(user_lst[i])
    rating_data.append(rating_lst[i])
print(user_lst[len(user_lst)-1])
rating_martix_coo = coo_matrix((rating_data, (row, col)), shape=(itemsFrame.shape[0], user_lst[len(user_lst)-1]+1))
rating_martix_csc = rating_martix_coo.tocsc()
rating_martix_csr = rating_martix_coo.tocsr()
start = 0
end = int(rating_martix_csc.shape[1] * 0.7)
sM_training = rating_martix_csc[:, start:end]
sM_testing =  rating_martix_csc[:, end:]

1748


In [ ]:
############### Build Tree #################
''' Save to file 'tree.pkl' '''
dtmodel_realdata = DecisionTreeModel(sM_training, depth_threshold = 10)
dtmodel_realdata.build_model()
o_tree = open('./tree_data_structure/tree.pkl', 'wb')
pickle.dump(dtmodel_realdata, o_tree)
o_tree.close()
############################################

rU Generation start:
0.00%
37.99%
75.98%
rU Generation DONE
bias, sum_cur_t, sum_2_cur_t Generation start:
0.00%
bias, sum_cur_t, sum_2_cur_t Generation DONE
Initiation DONE!
┃┃┃┃┃┃┃┃┃┏Current depth: 10        0.00%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        0.01%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        0.01%
┃┃┃┃┃┃┃┃┏Current depth: 9        0.01%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        0.02%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        0.02%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        0.02%
┃┃┃┃┃┃┃┃┏Current depth: 9        0.03%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        0.03%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        0.03%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        0.04%
┃┃┃┃┃┃┃┃┏Current depth: 9        0.04%
┃┃┃┃┃┃┃┏Current depth: 8        0.04%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        0.05%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        0.05%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        0.05%
┃┃┃┃┃┃┃┃┏Current depth: 9        0.06%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        0.06%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        0.06%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        0.07%
┃┃┃┃┃┃┃┃┏Current

┃┃┃┃┃┃┃┃┃┏Current depth: 10        0.71%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        0.72%
┃┃┃┃┃┃┃┃┏Current depth: 9        0.72%
┃┃┃┃┃┃┃┏Current depth: 8        0.72%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        0.73%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        0.73%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        0.73%
┃┃┃┃┃┃┃┃┏Current depth: 9        0.74%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        0.74%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        0.75%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        0.75%
┃┃┃┃┃┃┃┃┏Current depth: 9        0.75%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        0.76%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        0.76%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        0.76%
┃┃┃┃┃┃┃┃┏Current depth: 9        0.77%
┃┃┃┃┃┃┃┏Current depth: 8        0.77%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        0.77%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        0.78%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        0.78%
┃┃┃┃┃┃┃┃┏Current depth: 9        0.78%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        0.79%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        0.79%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        0.79%
┃┃┃┃┃┃┃┃┏Current depth: 9       

┃┃┃┃┃┃┃┃┃┏Current depth: 10        1.42%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        1.42%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        1.42%
┃┃┃┃┃┃┃┃┏Current depth: 9        1.43%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        1.43%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        1.43%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        1.44%
┃┃┃┃┃┃┃┃┏Current depth: 9        1.44%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        1.44%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        1.45%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        1.45%
┃┃┃┃┃┃┃┃┏Current depth: 9        1.45%
┃┃┃┃┃┃┃┏Current depth: 8        1.46%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        1.46%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        1.46%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        1.47%
┃┃┃┃┃┃┃┃┏Current depth: 9        1.47%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        1.47%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        1.48%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        1.48%
┃┃┃┃┃┃┃┃┏Current depth: 9        1.48%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        1.49%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        1.49%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        1.49%
┃┃┃┃┃┃┃┃┏Current depth: 9    

┃┃┃┃┃┃┃┃┃┏Current depth: 10        2.12%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        2.12%
┃┃┃┃┃┃┃┃┏Current depth: 9        2.12%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        2.13%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        2.13%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        2.13%
┃┃┃┃┃┃┃┃┏Current depth: 9        2.14%
┃┃┃┃┃┃┃┏Current depth: 8        2.14%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        2.14%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        2.15%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        2.15%
┃┃┃┃┃┃┃┃┏Current depth: 9        2.15%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        2.16%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        2.16%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        2.16%
┃┃┃┃┃┃┃┃┏Current depth: 9        2.17%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        2.17%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        2.17%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        2.18%
┃┃┃┃┃┃┃┃┏Current depth: 9        2.18%
┃┃┃┃┃┃┃┏Current depth: 8        2.18%
┃┃┃┃┃┃┏Current depth: 7        2.19%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        2.19%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        2.19%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        2

┃┃┃┃┃┃┃┃┃┏Current depth: 10        2.84%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        2.85%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        2.85%
┃┃┃┃┃┃┃┃┏Current depth: 9        2.85%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        2.86%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        2.86%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        2.86%
┃┃┃┃┃┃┃┃┏Current depth: 9        2.87%
┃┃┃┃┃┃┃┏Current depth: 8        2.87%
┃┃┃┃┃┃┏Current depth: 7        2.87%
┃┃┃┃┃┏Current depth: 6        2.88%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        2.88%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        2.88%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        2.89%
┃┃┃┃┃┃┃┃┏Current depth: 9        2.89%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        2.89%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        2.90%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        2.90%
┃┃┃┃┃┃┃┃┏Current depth: 9        2.90%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        2.91%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        2.91%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        2.91%
┃┃┃┃┃┃┃┃┏Current depth: 9        2.92%
┃┃┃┃┃┃┃┏Current depth: 8        2.92%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        2.92%


...

┃┃┃┃┃┃┃┃┃┏Current depth: 10        52.65%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        52.65%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        52.65%
┃┃┃┃┃┃┃┃┏Current depth: 9        52.66%
┃┃┃┃┃┃┃┏Current depth: 8        52.66%
┃┃┃┃┃┃┏Current depth: 7        52.66%
┃┃┃┃┃┏Current depth: 6        52.67%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        52.67%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        52.67%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        52.68%
┃┃┃┃┃┃┃┃┏Current depth: 9        52.68%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        52.68%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        52.69%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        52.69%
┃┃┃┃┃┃┃┃┏Current depth: 9        52.69%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        52.70%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        52.70%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        52.70%
┃┃┃┃┃┃┃┃┏Current depth: 9        52.71%
┃┃┃┃┃┃┃┏Current depth: 8        52.71%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        52.71%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        52.72%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        52.72%
┃┃┃┃┃┃┃┃┏Current depth: 9        52.72%
┃┃┃┃┃┃┃┃┃┏Current

┃┃┃┃┃┃┃┃┃┏Current depth: 10        53.35%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        53.36%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        53.36%
┃┃┃┃┃┃┃┃┏Current depth: 9        53.36%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        53.37%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        53.37%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        53.37%
┃┃┃┃┃┃┃┃┏Current depth: 9        53.38%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        53.38%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        53.38%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        53.39%
┃┃┃┃┃┃┃┃┏Current depth: 9        53.39%
┃┃┃┃┃┃┃┏Current depth: 8        53.39%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        53.40%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        53.40%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        53.40%
┃┃┃┃┃┃┃┃┏Current depth: 9        53.41%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        53.41%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        53.41%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        53.42%
┃┃┃┃┃┃┃┃┏Current depth: 9        53.42%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        53.42%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        53.43%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        53.43%
┃┃┃┃┃

┃┃┃┃┃┃┃┃┃┏Current depth: 10        54.05%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        54.06%
┃┃┃┃┃┃┃┃┏Current depth: 9        54.06%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        54.06%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        54.07%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        54.07%
┃┃┃┃┃┃┃┃┏Current depth: 9        54.07%
┃┃┃┃┃┃┃┏Current depth: 8        54.08%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        54.08%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        54.08%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        54.09%
┃┃┃┃┃┃┃┃┏Current depth: 9        54.09%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        54.09%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        54.10%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        54.10%
┃┃┃┃┃┃┃┃┏Current depth: 9        54.11%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        54.11%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        54.11%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        54.12%
┃┃┃┃┃┃┃┃┏Current depth: 9        54.12%
┃┃┃┃┃┃┃┏Current depth: 8        54.12%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        54.13%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        54.13%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        54.13%
┃┃┃┃┃┃┃┃

┃┃┃┃┃┃┃┃┃┏Current depth: 10        54.77%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        54.77%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        54.78%
┃┃┃┃┃┃┃┃┏Current depth: 9        54.78%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        54.78%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        54.79%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        54.79%
┃┃┃┃┃┃┃┃┏Current depth: 9        54.79%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        54.80%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        54.80%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        54.80%
┃┃┃┃┃┃┃┃┏Current depth: 9        54.81%
┃┃┃┃┃┃┃┏Current depth: 8        54.81%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        54.81%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        54.82%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        54.82%
┃┃┃┃┃┃┃┃┏Current depth: 9        54.82%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        54.83%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        54.83%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        54.83%
┃┃┃┃┃┃┃┃┏Current depth: 9        54.84%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        54.84%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        54.84%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        54.85%
┃┃┃┃┃

┃┃┃┃┃┃┃┃┃┏Current depth: 10        55.50%
┃┃┃┃┃┃┃┃┏Current depth: 9        55.50%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        55.51%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        55.51%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        55.51%
┃┃┃┃┃┃┃┃┏Current depth: 9        55.52%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        55.52%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        55.52%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        55.53%
┃┃┃┃┃┃┃┃┏Current depth: 9        55.53%
┃┃┃┃┃┃┃┏Current depth: 8        55.53%
┃┃┃┃┃┃┏Current depth: 7        55.54%
┃┃┃┃┃┏Current depth: 6        55.54%
┃┃┃┃┏Current depth: 5        55.54%
┃┃┃┏Current depth: 4        55.55%
┃┃┏Current depth: 3        55.55%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        55.55%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        55.56%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        55.56%
┃┃┃┃┃┃┃┃┏Current depth: 9        55.56%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        55.57%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        55.57%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        55.58%
┃┃┃┃┃┃┃┃┏Current depth: 9        55.58%
┃┃┃┃┃┃┃┃┃┏Current depth: 10        